In [2]:
import os
import numpy as np
import pandas as pd
import rasterio as rio
import rioxarray as rxr
import warnings
import matplotlib.pyplot as plt
import xarray
import datetime
import gc;gc.enable()
from collections import defaultdict
warnings.simplefilter('ignore')

In [3]:
filebase = "20180201T191000_maiac_la_0.hdf"
filename = "../data/train/maiac/2018/" + filebase

os.path.exists(filename)

True

In [4]:
data = rxr.open_rasterio(filename, masked=True)

In [ ]:
[
    [-117.3948356552278, 33.98201108613195], 
    [-117.3948356552278, 34.01924766510738],
    [-117.4397514194338, 34.01924766510738],
    [-117.4397514194338, 33.98201108613195],
    [-117.3948356552278, 33.98201108613195]
]

In [39]:
geometries = [
        {
            'type': 'Polygon',
            'coordinates': [
                [
    [-117.9338248256995, 33.79558357488509], 
    [-117.9338248256995, 33.83290166381627], 
    [-117.9787405899055, 33.83290166381627], 
    [-117.9787405899055, 33.79558357488509],
    [-117.9338248256995, 33.79558357488509]
    ],
        ]
        }
    ]
clipped_0 = data[0].rio.clip(geometries, crs=4326)

In [31]:
data[0]

<xarray.Dataset>
Dimensions:            (y: 1200, x: 1200, band: 4)
Coordinates:
  * y                  (y) float64 4.447e+06 4.446e+06 ... 3.337e+06 3.336e+06
  * x                  (x) float64 -1.112e+07 -1.112e+07 ... -1.001e+07
  * band               (band) int32 1 2 3 4
    spatial_ref        int32 0
Data variables:
    Optical_Depth_047  (band, y, x) float32 ...
    Optical_Depth_055  (band, y, x) float32 ...
    AOD_Uncertainty    (band, y, x) float32 ...
    FineModeFraction   (band, y, x) float32 ...
    Column_WV          (band, y, x) float32 ...
    AOD_QA             (band, y, x) float32 ...
    AOD_MODEL          (band, y, x) float32 ...
    Injection_Height   (band, y, x) float32 ...
Attributes: (12/85)
    ADDITIONALLAYERS:                   5
    ALGORITHMPACKAGEACCEPTANCEDATE:     TBD
    ALGORITHMPACKAGEMATURITYCODE:       Preliminary
    ALGORITHMPACKAGENAME:               MOD_PR19
    ALGORITHMPACKAGEVERSION:            2.0
    ASSOCIATEDINSTRUMENTSHORTNAME.1:    MODIS
    ...                                 ...
    TileID:                             51008005
    TOTALADDITIONALOBSERVATIONS:        767425
    TOTALOBSERVATIONS:                  8024888
    VERSIONID:                          6
    VERTICALTILENUMBER:                 5
    WESTBOUNDINGCOORDINATE:             -130.540728914638

In [32]:
latitude = np.linspace(clipped_0.SOUTHBOUNDINGCOORDINATE, clipped_0.NORTHBOUNDINGCOORDINATE, num=1200)
longitude = np.linspace(clipped_0.WESTBOUNDINGCOORDINATE, clipped_0.EASTBOUNDINGCOORDINATE, num=1200)

In [33]:
from shapely.geometry import Point, Polygon

points = []
for lt in latitude:
    for lg in longitude:
        points.append(Point(lg, lt))

In [35]:
poly = Polygon([
    [-117.9338248256995, 33.79558357488509], 
    [-117.9338248256995, 33.83290166381627], 
    [-117.9787405899055, 33.83290166381627], 
    [-117.9787405899055, 33.79558357488509],
    [-117.9338248256995, 33.79558357488509]
    ])

isthere = []
for el in points:
    isthere.append(poly.contains(el))

In [43]:
clipped_0['AOD_MODEL']

<xarray.DataArray 'AOD_MODEL' (band: 4, y: 5, x: 9)>
array([[[nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan]],

       [[nan, nan, nan, nan,  2.,  2.,  2.,  2.,  2.],
        [nan, nan, nan,  2.,  2.,  2.,  2.,  2., nan],
        [nan, nan,  2.,  2.,  2.,  2.,  2., nan, nan],
        [nan,  2.,  2.,  2.,  2., nan, nan, nan, nan],
        [ 2.,  2.,  2.,  2., nan, nan, nan, nan, nan]],

       [[nan, nan, nan, nan,  2.,  2.,  2.,  2.,  2.],
        [nan, nan, nan,  2.,  2.,  2.,  2.,  2., nan],
        [nan, nan,  2.,  2.,  2.,  2.,  2., nan, nan],
        [nan,  2.,  2.,  2.,  2., nan, nan, nan, nan],
        [ 2.,  2.,  2.,  2., nan, nan, nan, nan, nan]],

       [[nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan]]], dtype=float32)
Coordinates:
  * y            (y) float64 3.762e+06 3.761e+06 3.76e+06 3.759e+06 3.758e+06
  * x            (x) float64 -1.09e+07 -1.09e+07 ... -1.089e+07 -1.089e+07
  * band         (band) int32 1 2 3 4
    spatial_ref  int32 0
Attributes:
    scale_factor:  1.0
    add_offset:    0.0
    long_name:     Regional background model used